<h1 align=left><font size = 5>Week 3-Segmenting and Clustering Neighborhoods in Toronto</font></h1>
<h1 align=left><font size = 4>Part 1-Build data frame from Wikipedia</font></h1>

In [29]:
# Load dependencies

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


# Scrape neighborhood information from Wikipedia

In [36]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,'lxml')
my_table = soup.find('table',{'class':'wikitable sortable'})
neigh=my_table.findAll('tr')

# Build Pandas dataframe from Wiki table 

In [43]:
# Empty dataframe with column names

df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [44]:
# Parse the table into a Pandas dataframe
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

i=0
for i in range(len(neigh)):
    if (neigh[i].contents[3].text != 'Not assigned' or neigh[i].contents[5].text != 'Not assigned\n'):
        df.loc[i]=[neigh[i].contents[1].text,neigh[i].contents[3].text,neigh[i].contents[5].text]
        i=i+1
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')
df.drop(df[df.Borough == 'Not Assigned'].index,inplace=True)
df.drop(df[df.PostalCode == 'Postcode'].index,inplace=True)

for i in range(len(df)):
    if df.iloc[i]['Neighborhood'] == 'Not assigned':
        df.iloc[i]['Neighborhood'] = df.iloc[i]['Borough']

In [51]:
df.head(15)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


# Clean-up dataframe

In [47]:
# Group neighborhoods by PostalCode

grp=df.groupby(['PostalCode','Borough'])
df1=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

i=0
for name,group in grp:
    df1.loc[i]=[list(name)[0],list(name)[1],group['Neighborhood'].tolist()]
    i=i+1
df1['Neighborhood'] = df1['Neighborhood'].apply(lambda x: ','.join(map(str, x)))

In [50]:
df1.head(15)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [49]:
df1.shape

(103, 3)